Миллионы людей ежедневно пользуются поиском по картинкам в Яндексе. 
Команда Яндекса прикладывает много усилий чтобы сделать сервис более полезным и удобным. 
Но для этого нужно хорошо понимать, какие задачи решают пользователи. К примеру, есть гипотеза, что интересы людей, ищущих картинки на мобильных устройствах и на десктопных компьютерах, заметно отличаются.
Вам нужно проверить эту гипотезу, сравнив запросы на разных платформах.

В файле data.json лежит выборка запросов к Яндекс Картинкам за несколько недель.
Каждый запрос описан словарем со следующими параметрами:
```
    query - текст запроса
    ts - timestamp запроса
    platform - платформа (touch - мобильный устройства, desktop - компьютеры)
```
Во время интервью Вам будет предложено ответить на ряд вопросов на основе имеющихся данных.

В ходе работы над задачей Вам будет разрешено использовать любые дополнительные материалы, поисковые системы и т.п.

**Примеры вопросов, которые могут быть заданы**
1. Выведите топ-10 самых частотных запросов в каждой платформе (desktop, touch). Какие отличия Вы видите?
2. Посмотрите, чем отличается трафик запросов в течение дня. Как можно объяснить отличие?
3. Выделите тематики запросов, контрастные для мобильных и компьютеров. Являются ли различия статистически значимыми?

### топ-10 самых частотных запросов в каждой платформе

In [178]:
# !pip list
# !pip install pandas

import pandas as pd
import numpy as np
from datetime import datetime

In [179]:
#читаем данные

data = pd.read_json('data.json')
data.head(5)

,query,ts,platform
0,порно фото,1631786697,touch
1,малевич картины,1631806465,desktop
2,Секс,1631781583,touch
3,с днём рождения лена,1631771563,touch
4,зверополис порно,1631787599,touch


In [180]:
# описательная сатистика

data.describe(include=[object])

,query,platform
count,1203824,1203824
unique,25737,2
top,порно,touch
freq,9082,792258


In [181]:
# описательная сатистика

data.describe(include=[np.number])

,ts
count,1.203824e+06
mean,1.631374e+09
std,5.292754e+05
min,1.630444e+09
25%,1.630929e+09
50%,1.631380e+09
75%,1.631818e+09
max,1.632258e+09


In [182]:
# фильтрация

data[data['query'] == 'малевич картины'].count()

query       22
ts          22
platform    22
dtype: int64

In [185]:
# применение функции к столбцу

data['date'] = data['ts'].map(lambda x: datetime.utcfromtimestamp(x).date().strftime('%Y-%m-%d %H:%M:%S'))
data

,query,ts,platform,date
0,порно фото,1631786697,touch,2021-09-16 00:00:00
1,малевич картины,1631806465,desktop,2021-09-16 00:00:00
2,Секс,1631781583,touch,2021-09-16 00:00:00
3,с днём рождения лена,1631771563,touch,2021-09-16 00:00:00
4,зверополис порно,1631787599,touch,2021-09-16 00:00:00
...,...,...,...,...
1203819,бабочка,1631475653,touch,2021-09-12 00:00:00
1203820,все республики россии,1631462934,touch,2021-09-12 00:00:00
1203821,кухня в стиле лофт,1631432252,touch,2021-09-12 00:00:00
1203822,топор,1631449347,desktop,2021-09-12 00:00:00


In [186]:
# группировка и соритровка

counts = data.groupby(['platform', 'query']).count().sort_values(['platform', 'ts'], ascending=False).reset_index()
counts

,platform,query,ts,date
0,touch,порно,7740,7740
1,touch,секс,6348,6348
2,touch,с днём рождения женщине,4903,4903
3,touch,с днём рождения,3967,3967
4,touch,с днём рождения мужчине,3623,3623
...,...,...,...,...
49495,desktop,языки любви,1,1
49496,desktop,яна кошкина до пластики,1,1
49497,desktop,яндекс картинки загрузить,1,1
49498,desktop,япончик и патриарх,1,1


In [187]:
counts[counts['platform'] == 'touch'].head(10)
# дни рождения и секс

,platform,query,ts,date
0,touch,порно,7740,7740
1,touch,секс,6348,6348
2,touch,с днём рождения женщине,4903,4903
3,touch,с днём рождения,3967,3967
4,touch,с днём рождения мужчине,3623,3623
5,touch,хентай,3157,3157
6,touch,с днем рождения,3005,3005
7,touch,xxx,2429,2429
8,touch,Порно,2330,2330
9,touch,календарь 2021,2237,2237


In [188]:
counts[counts['platform'] == 'desktop'].head(10)
# образовательный контент, секс

,platform,query,ts,date
25544,desktop,календарь 2021,2804,2804
25545,desktop,таблица менделеева,2631,2631
25546,desktop,картинки,1647,1647
25547,desktop,порно,1342,1342
25548,desktop,английский алфавит,1293,1293
25549,desktop,обои на рабочий стол,1143,1143
25550,desktop,Одноклассники (социальная сеть),1116,1116
25551,desktop,секс,895,895
25552,desktop,таблица квадратов,877,877
25553,desktop,алфавит,874,874


### Трафик запросов в течение дня. Мобилки.

In [189]:
mobile = data[data['platform'] == 'touch'].reset_index()[['query', 'ts']]
mobile

,query,ts
0,порно фото,1631786697
1,Секс,1631781583
2,с днём рождения лена,1631771563
3,зверополис порно,1631787599
4,алабай собака фото,1631786645
...,...,...
792253,голые мужчины,1631396814
792254,бабочка,1631475653
792255,все республики россии,1631462934
792256,кухня в стиле лофт,1631432252


In [190]:
# какие у нас дни в датасете?

data['ts'].map(lambda x: datetime.utcfromtimestamp(x).date().strftime('%Y-%m-%d %H:%M:%S')).unique()

array(['2021-09-16 00:00:00', '2021-09-15 00:00:00',
       '2021-09-19 00:00:00', '2021-09-18 00:00:00',
       '2021-09-10 00:00:00', '2021-09-09 00:00:00',
       '2021-09-14 00:00:00', '2021-09-13 00:00:00',
       '2021-09-17 00:00:00', '2021-09-03 00:00:00',
       '2021-09-02 00:00:00', '2021-09-01 00:00:00',
       '2021-09-20 00:00:00', '2021-09-04 00:00:00',
       '2021-09-11 00:00:00', '2021-08-31 00:00:00',
       '2021-09-05 00:00:00', '2021-09-06 00:00:00',
       '2021-09-07 00:00:00', '2021-09-12 00:00:00',
       '2021-09-08 00:00:00', '2021-09-21 00:00:00'], dtype=object)

In [191]:
bundles = [
    '1900-01-01 00:00:00', 
    '1900-01-01 06:00:00', 
    '1900-01-01 10:00:00', 
    '1900-01-01 15:00:00', 
    '1900-01-01 18:00:00',
    '1900-01-01 21:00:00', 
]

In [192]:
def get_bundle_by_ts(ts, bundles):
    ts = datetime.utcfromtimestamp(ts).time().strftime('%Y-%m-%d %H:%M:%S')
    for i, item in enumerate(bundles):
        if ts < item:
            return i
    return len(bundles)

In [193]:
mobile['bundle'] = mobile['ts'].map(lambda x: get_bundle_by_ts(x, bundles))
mobile = mobile[['query', 'bundle']]
mobile

,query,bundle
0,порно фото,3
1,Секс,2
2,с днём рождения лена,1
3,зверополис порно,3
4,алабай собака фото,3
...,...,...
792253,голые мужчины,6
792254,бабочка,5
792255,все республики россии,4
792256,кухня в стиле лофт,2


In [194]:
groups = mobile.groupby(['bundle', 'query']).size().reset_index(name='count').sort_values(['bundle', 'count'], ascending=False)
groups

,bundle,query,count
121822,6,порно,507
123331,6,секс,283
125218,6,хентай,209
114783,6,Порно,163
113879,6,porno,147
...,...,...,...
20403,1,ярыгин станислав сергеевич,1
20405,1,ясень дерево,1
20408,1,яхико,1
20414,1,ящерица,1


In [195]:
for i in range(len(bundles) + 1):
    print('group ', i)
    print(groups[groups['bundle'] == i].head(10))
    print()

group  0
Empty DataFrame
Columns: [bundle, query, count]
Index: []

group  1
       bundle                    query  count
15347       1  с днём рождения женщине   1644
6404        1              доброе утро   1335
15281       1          с днём рождения   1114
15432       1  с днём рождения мужчине   1091
13562       1                    порно   1065
15133       1          с днем рождения   1029
6421        1     доброе утро картинки   1017
15042       1             с 1 сентября    883
15992       1                     секс    818
15564       1           с добрым утром    588

group  2
       bundle                    query  count
36052       2                    порно   1563
38160       2  с днём рождения женщине   1554
38094       2          с днём рождения   1361
38849       2                     секс   1305
38245       2  с днём рождения мужчине   1239
37947       2          с днем рождения    898
40662       2       таблица менделеева    613
21971       2                      xxx 

Откуда столько запросов на день рождения?

### Трафик запросов в течение дня. Десктоп.

In [196]:
desk = data[data['platform'] == 'desktop'].reset_index()[['query', 'ts']]
desk

,query,ts
0,малевич картины,1631806465
1,бактериофаг,1631816202
2,кадр из фильма,1631770837
3,картинки,1631807692
4,драцена,1631792011
...,...,...
411561,календарь 2021,1631420479
411562,майнкрафт,1631452568
411563,фон,1631456396
411564,снежный барс,1631470153


In [197]:
desk['bundle'] = desk['ts'].map(lambda x: get_bundle_by_ts(x, bundles))
desk = desk[['query', 'bundle']]
desk

,query,bundle
0,малевич картины,4
1,бактериофаг,5
2,кадр из фильма,1
3,картинки,4
4,драцена,3
...,...,...
411561,календарь 2021,1
411562,майнкрафт,3
411563,фон,3
411564,снежный барс,5


In [198]:
groups_desk = desk.groupby(['bundle', 'query']).size().reset_index(name='count').sort_values(['bundle', 'count'], ascending=False)
groups_desk

,bundle,query,count
91857,6,порно,74
89914,6,картинки,60
93071,6,таблица менделеева,55
89765,6,календарь 2021,50
87342,6,Девушка В Платье,44
...,...,...,...
13664,1,яхико,1
13665,1,яхта,1
13668,1,яшма,1
13671,1,яэ мико геншин,1


In [199]:
for i in range(len(bundles) + 1):
    print('group ', i)
    print(groups_desk[groups_desk['bundle'] == i].head(10))
    print()

group  0
Empty DataFrame
Columns: [bundle, query, count]
Index: []

group  1
       bundle                                            query  count
5238        1                                   календарь 2021    400
1419        1                  Одноклассники (социальная сеть)    184
11714       1                               таблица менделеева    177
5515        1                                         картинки    174
573         1                                    iso 9001 2015    152
5270        1  календарь на 2022 год с праздниками и выходными    145
9114        1                                            порно    142
10363       1                          с днём рождения женщине    129
2040        1                               английский алфавит    123
5251        1                                   календарь 2022    113

group  2
       bundle                                            query  count
20731       2                                   календарь 2021    905
295

На десктопе видна тенденция - образовательный контент повышает актуальность ближе к вечеру и спадает к ночи. К ночи увеличивается интерес к порно
Вопрос: зачем людям календарь, и почему в топах нет стримминговых сервисов (YT, VK...)?

### Контрастные тематики

Для мобилок - очень много поздравлений с днем рождения.
Для десктопа - очень много образовательной тематики.
Будем значит проводить 2 теста. 

Первый тест. На мобилки чаще приходят запросы, связанные с днем рождения. Идея 1 - собирать выборку по запросам. То есть первое значение - это число "с днем рождения женщине"... Все проще - у нас есть данные по дням за весь сентябрь 2021. 

In [202]:
data.head(5)

,query,ts,platform,date
0,порно фото,1631786697,touch,2021-09-16 00:00:00
1,малевич картины,1631806465,desktop,2021-09-16 00:00:00
2,Секс,1631781583,touch,2021-09-16 00:00:00
3,с днём рождения лена,1631771563,touch,2021-09-16 00:00:00
4,зверополис порно,1631787599,touch,2021-09-16 00:00:00


In [229]:
mobile_hb = data[
    (data['platform'] == 'touch') &
    (data['query'].str.contains('рождени'))
]
mobile_hb_for_test = mobile_hb.groupby('date').count().sort_values('date').reset_index()[['date', 'query']]
mobile_hb_for_test

,date,query
0,2021-08-31 00:00:00,113
1,2021-09-01 00:00:00,3053
2,2021-09-02 00:00:00,2685
3,2021-09-03 00:00:00,2791
4,2021-09-04 00:00:00,2573
5,2021-09-05 00:00:00,2681
6,2021-09-06 00:00:00,2694
7,2021-09-07 00:00:00,2727
8,2021-09-08 00:00:00,2755
9,2021-09-09 00:00:00,2788


In [232]:
desktop_hb = data[
    (data['platform'] == 'desktop') &
    (data['query'].str.contains('рождени'))
]
desktop_hb_for_test = desktop_hb.groupby('date').count().sort_values('date').reset_index()[['date', 'query']]
desktop_hb_for_test

,date,query
0,2021-08-31 00:00:00,10
1,2021-09-01 00:00:00,350
2,2021-09-02 00:00:00,275
3,2021-09-03 00:00:00,269
4,2021-09-04 00:00:00,203
5,2021-09-05 00:00:00,234
6,2021-09-06 00:00:00,330
7,2021-09-07 00:00:00,301
8,2021-09-08 00:00:00,307
9,2021-09-09 00:00:00,321


In [235]:
mobile_sample = mobile_hb_for_test['query']
desktop_sample = desktop_hb_for_test['query']

In [241]:
from scipy.stats import mannwhitneyu

stat, p = mannwhitneyu(mobile_sample, desktop_sample, alternative='two-sided') 
print(f"U-статистика: {stat}, p-значение: {p}")

U-статистика: 463.0, p-значение: 2.265976727164243e-07


In [242]:
# pvalue << 0.05, так что дней рождения реально больше на мобилках